<div id="teaser" style=' background-position:  right center; background-size: 00px; background-repeat: no-repeat; 
    padding-top: 20px;
    padding-right: 10px;
    padding-bottom: 170px;
    padding-left: 10px;
    border-bottom: 14px double #333;
    border-top: 14px double #333;' > 

   
   <div style="text-align:center">
    <b><font size="6.4">Exploratory analysis of raw data using unsupervised learning</font></b>    
  </div>
    
<p>
 created by:
 Luigi Sbailo<sup>1</sup> 
 and Luca Ghiringhelli<sup>1</sup> <br><br>
   
<sup>1</sup> Fritz Haber Institute of the Max Planck Society, Faradayweg 4-6, D-14195 Berlin, Germany <br>

  
<div> 
<img  style="float: left;" src="assets/exploratory_analysis/Logo_MPG.png" width="200"> 
<img  style="float: right;" src="assets/exploratory_analysis/Logo_NOMAD.png" width="250">
</div>
</div>

In this tutorial we use unsupervised learning for a preliminary exploration of materials science data. More specifically, we analyze 82 octet binary materials known to crystallize in zinc blende (ZB) and rocksalst (RS) structures. Our aim is to identify the right strategy to facilitate the visualization and characterization of unlabeled data. As a first step in our data analysis, we would like to detect whether data points can be classified into different  clusters, where each cluster is aimed to group together objects that share similar features. With an explorative analysis we would like to visualize the structure and spatial displacement of the clusters, but when the feature space is higlhly multidimensional such visualization is directly not possible. Hence, we project the feature space into a two-dimensional manifold that can be  visualized. To avoid losing relevant information, the embedding into a lower dimensional manifold must be performed while preserving the most informative features in the original space. Below we introduce into different clustering and embedding methods, which can be combined to obtain different visualizations of our dataset.

# Introduction to clustering

Cluster analysis is performed to group together data points that are more similar to each other in comparison with points belonging to other clusters. Clustering can be achieved by means of many different algorithms, each with proper characteristics and input parameters. The choice of the specific clustering algorithms to be used depends on the individual data set analyzed, and, once an optimal algorithm has been chosen, it is often necessary to iteratively modify the input parameters until results achieve the desired properties. We focus on three distinct algorithms as described below.
- ___k_-means__ partitions the data set into _k_ clusters, where each data point belongs to the cluster with the nearest mean. This partition ultimately minimizes the within-cluster variance to find the most compact partitioning of the data set. _K_-means uses an iterative refinement technique that is fast and scalable, but if falls in local minima. Thus, the algorithm is iterated multiple times with different initial conditions and the best outcome is finally chosen. Drawbacks of this algorithm are that the number of clusters _k_ is an input parameter which must be known in advance and clusters are convex shaped.
- Density-based spatial clustering of applications with noise (__DBSCAN__) is an algorithm that, without knowing the exact number of clusters, groups points that are close to each other leaving outliers marked as noise and not defined in any cluster. In this algorithm a neighborood distance _$\epsilon$_  and a number of points _min-samples_ are used to determine if a point belongs to a cluster: if the point has a number _min-samples_ of other points  within the distance _$\epsilon$_ is marked as core point and belongs to a cluster; otherwise, the point is marked as noise. This algorithm is fast and clusters can assume any shape, but the outcome depends on the initial order of the data points.
- __Hierarchical clustering__ builds a hierarchy of clusters with a bottom-up (__agglomerative__) or top-down (__divisive__) approach. In a bottom-up approach, that we deploy below, starting with all data points placed in its own cluster, different pairs of clusters are iteratively merged together where the decision of the clusters to be merged is determined in a greedy manner. This is iterated until all points are grouped within one cluster, and the resulting hierarchy of clusters is presentend in a dendogram. Given a distance thereshold it is possible to avoid merging of clusters when outside this distance, this stops the algorithm when no more mergings are possible. The algorithm then returns a certain number of clusters as a function of the threshold distance . An advantage of this algorithm is that the construction of dendroids allows for a visual inspection of the clustering, but hierarchical clustering is considerably slower than the other algorithms discussed above and not well suited for big data.


# Introduction to embedding

Visualization of a dataset is not possible when it is defined in a highly multidimensional space. To facilitate visualization of inner structures in the dataset, we reduce the dimensionality of the system with methodologies specifically developed to avoid losing critical information, which are introduced below.
- Principal component analysis (__PCA__) is a linear projection method that seeks for an orthogonal transformation of the dataset so as to render the variables of the dataset uncorrelated. The dimensionality reduction is then performed onto the features with highest variance to preserve as much information as possible. This is a deterministic but linear method, that fails to catch non linear correlations.
- Multi-dimensional scaling (__MDS__) constructs a pairwise distance matrix in the original space, and seeks a low-dimensional representation that preserves the original distances as much as possible. This method tends to preserve local structures better than global structures and scales badly with the number of the data points. 
- T-distributed Stochastic Neighbor Embedding (__t-SNE__) is a non-linear dimensionality reduction method that converts similarities between data points to joint probabilities and minimizes the Kullback-Leibler divergence between the joint probabilities of the embedding and the original space. The cost function is not convex and results depend on the inizialization. Non linear effects in this method might occasionally produce misleading results, a fine parameter tuning and several iterations of the method are then recommended.


# Import required modules

In [1]:
from ase.io import read
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [2]:
btn_PCA = widgets.Button(description='PCA')
btn_MDS = widgets.Button(description='MDS')
btn_tSNE = widgets.Button(description='t-SNE')


def btn_eventhandler(obj):
    method = str(obj.description)
    print(method)
    if (method == 'PCA'):
        transformed_data = PCA(n_components=2).fit_transform(data)
    elif (method == 'MDS'):
        transformed_data = MDS (n_components=2).fit_transform(data)
    elif (method == 't-SNE'):
        transformed_data = TSNE (n_components=2).fit_transform(data)
    df['x_emb']=transformed_data[:,0]
    df['y_emb']=transformed_data[:,1]
    display(px.scatter(df,x='x_emb',y='y_emb',color='labels',hover_data=df[features], hover_name=df.index ))
        

btn_PCA.on_click(btn_eventhandler)
btn_MDS.on_click(btn_eventhandler)
btn_tSNE.on_click(btn_eventhandler)

# Get the data
Let us load the data from the file data/data.pkl into a data frame. The data was downloaded from the NOMAD archive and the NOMAD atomic data collection. It consists of RS-ZB energy differences (in eV/atom) of the 82 octet binary compounds, structure objects containing the atomic positions of the materials and properties of the atomic constituents. The following atomic features are considered:

<div >
   <ul>
      <li>Z:  atomic number</li>
      <li>period: period in the periodic table</li>
      <li>IP: ionization potential</li>
      <li>EA: electron affinity</li>      
      <li>E_HOMO: energy of the highest occupied atomic orbital</li>
      <li>E_LUMO: energy of the lowest unoccupied atomic orbital</li>   
      <li>r_(s, p, d): radius where the radial distribution of s, p or d orbital has its maximum.</li>
   </ul>
</div>

In [3]:
# load data
RS_structures = read("data/RS_structures.xyz", index=':')
ZB_structures = read("data/ZB_structures.xyz", index=':')

def generate_table(RS_structures, ZB_structures):

    for RS, ZB in zip(RS_structures, ZB_structures):
        energy_diff = RS.info['energy'] - ZB.info['energy']
        min_struc_type = 'RS' if energy_diff < 0 else 'ZB'
        struc_obj_min = RS if energy_diff < 0 else ZB

        yield [RS.info['energy'], ZB.info['energy'],
               energy_diff, min_struc_type,
               RS.info['Z'], ZB.info['Z'],
               RS.info['period'], ZB.info['period'],
               RS.info['IP'], ZB.info['IP'],
               RS.info['EA'], ZB.info['EA'],
               RS.info['E_HOMO'], ZB.info['E_HOMO'],
               RS.info['E_LUMO'], ZB.info['E_LUMO'],
               RS.info['r_s'], ZB.info['r_s'],
               RS.info['r_p'], ZB.info['r_p'],
               RS.info['r_d'], ZB.info['r_d'],
               RS, ZB, struc_obj_min]
        
    
df = pd.DataFrame(
    generate_table(RS_structures, ZB_structures),
    columns=['energy_RS', 'energy_ZB', 
             'energy_diff', 'min_struc_type', 
             'Z(A)', 'Z(B)', 
             'period(A)', 'period(B)', 
             'IP(A)', 'IP(B)', 
             'EA(A)', 'EA(B)', 
             'E_HOMO(A)', 'E_HOMO(B)', 
             'E_LUMO(A)', 'E_LUMO(B)', 
             'r_s(A)', 'r_s(B)', 
             'r_p(A)', 'r_p(B)', 
             'r_d(A)', 'r_d(B)',
             'struc_obj_RS', 'struc_obj_ZB', 'struc_obj_min'],
    index=list(RS.get_chemical_formula() for RS in RS_structures)
)

df.insert(0,'x_emb',np.zeros(len(df)))
df.insert(0,'y_emb',np.zeros(len(df)))
df.insert(0,'labels',np.zeros(len(df)))

We select which features will be used for the clustering and embedding methods. The complexity of the problem clearly is reduced with lowering the number of features that are considered, and an accurate selection of the features to be processed can imporove the quality of the results. To find the most meaningful results it is sometimes necessary to iterate training while considering different features at each iteration.  

In [4]:
features = []
features.append('energy_diff')
# features.append('energy_RS')
# features.append('energy_ZB')
# features.append('Z(A)')
# features.append('Z(B)')
# features.append('IP(A)')
# features.append('IP(B)')
features.append('EA(A)')
features.append('EA(B)')
features.append('E_HOMO(A)')
features.append('E_HOMO(B)')
features.append('E_LUMO(A)')
features.append('E_LUMO(B)')
features.append('r_s(A)')
features.append('r_s(B)')
features.append('r_p(A)')
features.append('r_p(B)')
features.append('r_d(A)')
features.append('r_d(B)')



Machine learning algorithms can improve their performance if data is standardized. In fact, training can be biased towards dimensions presenting higher absolute values, or outliers can undermine the learning capabilites  of the algorithm. Hence, we standardize our dataset by subtracting the mean value and dividing it by the standard deviation for each variable. 

In [5]:
data = df.filter(items=features).to_numpy()
data = preprocessing.scale(data)

---
# K-Means

K-means requires the knowledge of the number of clusters and clustering depends on the initial conditions, hence the algorithm is iterated,  up to _max\_iter_ times, with different initial conditions until convergence. As initial guess we seek for 2 clusters and run the algorithm up to 200 iterations. 

In [6]:
n_clusters = 2
max_iter = 200
cluster_labels = KMeans (n_clusters=n_clusters, max_iter=max_iter).fit_predict(data)
df['labels']=cluster_labels
df["labels"]=df["labels"].astype(str)
display(btn_PCA,btn_MDS,btn_tSNE)

Button(description='PCA', style=ButtonStyle())

Button(description='MDS', style=ButtonStyle())

Button(description='t-SNE', style=ButtonStyle())

Could you identify and visualize two distinct clusters within your data? If not the number of clusters must be changed among the input parameters. You can also run the k-means clustering again and select only 1 as _max\_iter_ , which means that the first output is taken as optimal result. Try this again and compare the results, does the output change at each iteration? What happens instead if the number is much larger?

Note that also MDS and t-SNE are stochastich algorithms, so it might be worth iterate also the embedding to find a more satisfying result.

---
# DBSCAN

The most relevant parameter of DBSCAN is the maximum distance $\epsilon$ that determines the extent of the cluster and whether a point is considered as noise.

In [7]:
eps = 3
min_samples=5
cluster_labels = DBSCAN(eps=eps, min_samples=min_samples).fit_predict(data)
df['labels']=cluster_labels
df["labels"]=df["labels"].astype(str)
display(btn_PCA,btn_MDS,btn_tSNE)

Button(description='PCA', style=ButtonStyle())

Button(description='MDS', style=ButtonStyle())

Button(description='t-SNE', style=ButtonStyle())

Is the number of clusters that you have just found the same as the one that you used in k-means? Can you spot the noise in the visualization? What happens lowering the maximal distance $\epsilon$?

MDS seeks for an embedding that tries to preserve pairwise distances. Can you notice that noise is distant from other clusters? Does the same happen with t-SNE? 

---
# Hierarchical agglomerative clustering

In a hierarchical agglomerative clustering different clusters are iteratively merged if their distance is lower than a _distance\_threshold_. The number of clusters obtained is a function of this threshold.

In [8]:
distance_threshold=15
cluster_labels = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold).fit_predict(data)
df['labels']=cluster_labels
df["labels"]=df["labels"].astype(str)
display(btn_PCA,btn_MDS,btn_tSNE)

Button(description='PCA', style=ButtonStyle())

Button(description='MDS', style=ButtonStyle())

Button(description='t-SNE', style=ButtonStyle())

Several different possible metrics can be used for the linkage criterium. As a default option, we have used the Ward distance, which minimizes the sum of squared differences within all clusters. This has some similarities with the objective function of _k-means_, but tackled differently. By tuning the parameters, can you find the same results using the two different methodologies?

Now try again including different features. Is the classification obtained identical?
You can hover over the different plots and explore the classification of the materials.
Can you identify  meaningful clusters that group together materials that share similar properties? What clustering and embedding methods provide the most meaningful visualization of the data set?